In [31]:
# !pip install polars
# !pip install vaex
# !pip install duckdb
# !pip install fastparquet

In [1]:
from pathlib import Path

import pandas as pd
import vaex as vx
import polars as pl
import duckdb

from IPython.display import Markdown, display

In [2]:
def printmd(string):
    display(Markdown(string))

In [3]:
rawDataPath = Path('../data/raw')
csvFile = rawDataPath / 'large.csv'
parquetFile = rawDataPath / 'large.parquet'
csvFile.exists() and parquetFile.exists()

True

In [4]:
df.head()

NameError: name 'df' is not defined

In [ ]:
# df = pl.concat([pl.read_csv(csvFile) for _ in range(100)],axis=1)
# df.to_csv(rawDataPath / 'large.csv')
# df.to_parquet(rawDataPath / 'large.parquet')
# df.head()

In [5]:
dfcsv = pl.read_csv(rawDataPath / 'large.csv')
dfpar = pl.read_parquet(rawDataPath / 'large.parquet')

In [6]:
dfcsv.shape

(19900, 3926)

In [7]:
dfpar.shape

(19900, 3926)

In [ ]:
!du -h ../data/raw/large.*

The time benchmarks here should be taken with a pinch of salt. These libraries have many distinct features beyond that captured here. For example:
- DuckDB is a very lightweight dependancy that makes it highly portable
- DuckDB has aimed for a wide testing base rather than optimising for performance
- Vaex uses memory-mapping that makes it highly effective on large datasets
- etc

In [11]:
# to start an in-memory database
con = duckdb.connect(database=':memory:')

# Compare read time for a CSV

In [40]:
printmd("**Pandas with default C parser**")
%timeit -n 1 -r 1 pd.read_csv(csvFile)

Pandas with default C parser

48.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [41]:
printmd("**Pandas with pyarrow parser**")
%timeit -n 1 -r 1 pd.read_csv(csvFile,engine='pyarrow')

**Pandas with pyarrow parser**

1min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [42]:
printmd("**Vaex**")
%timeit -n 1 -r 1 vx.read_csv(csvFile)

**Vaex**

49.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [43]:
printmd("**Polars**")
%timeit -n 1 -r 1 pl.read_csv(csvFile)

**Polars**

22.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [44]:
printmd("**DuckDB**")
%timeit -n 1 -r 1 con.execute(f"SELECT * FROM '{csvFile}';").fetchdf()

**DuckDB**

39.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Compare read time for a Parquet file

In [45]:
printmd("**Pandas with default parquet reader**")
%timeit -n 1 -r 1 pd.read_parquet(parquetFile)

**Pandas**

7.65 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [46]:
printmd("**Pandas with pyarrow parquet reader**")
%timeit -n 1 -r 1 pd.read_parquet(parquetFile,engine='pyarrow')

**Pandas with pyarrow parquet reader**

990 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [47]:
printmd("**Pandas with fastparquet reader**")
%timeit -n 1 -r 1 pd.read_parquet(parquetFile,engine='fastparquet')

**Pandas with fastparquet reader**

1.25 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [48]:
printmd("**Vaex**")
%timeit -n 1 -r 1 vx.open(parquetFile)

**Vaex**

703 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [56]:
printmd("**Polars**")
%timeit -n 1 -r 3 pl.read_parquet(parquetFile)

**Polars**

198 ms ± 55.9 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [55]:
printmd("**Polars with pyarrow**")
%timeit -n 1 -r 3 pl.read_parquet(parquetFile,use_pyarrow=True)

**Polars with pyarrow**

1.36 s ± 456 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [51]:
printmd("**DuckDB**")
%timeit -n 1 -r 1 con.execute(f"SELECT * FROM '{parquetFile}';").fetchdf()

**DuckDB**

12.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [70]:
def groupbyPandas(df:pd.DataFrame):
    return df.groupby('z').mean()
df = pl.read_parquet(parquetFile).to_pandas()
gbPandas = groupbyPandas(df=df)


In [79]:
def groupbyPolars(df:pl.DataFrame):
    gb = df.lazy().groupby('z').agg([pl.mean(df.columns[1:])]).sort("z", reverse=False)
    return gb.collect()
df = pl.read_parquet(parquetFile)
gbPolars = groupbyPolars(df=df)
pd.testing.assert_frame_equal(gbPandas,gbPolars.to_pandas().set_index('z'))

In [6]:
df.head()

#,z,0:June:2020,1:June:2020,2:July:2020,3:June:2021,4:June:2021,5:June:2021,6:April:2016,7:March:2018,8:May:2005,9:August:2016,10:June:2002,11:April:2002,12:November:2001,13:November:2001,14:November:2001,15:January:2003,16:December:2002,17:December:2002,18:December:2002,19:November:2002,20:November:2002,21:November:2002,22:October:2002,23:October:2002,24:October:2002,25:September:2002,26:September:2002,27:September:2002,28:August:2002,29:August:2002,30:August:2002,31:July:2002,32:July:2002,33:July:2002,34:June:2002,35:June:2002,36:June:2002,37:May:2002,38:April:2002,39:April:2002,40:April:2002,41:March:2002,42:March:2002,43:January:2002,44:January:2002,45:January:2002,46:December:2001,47:December:2001,48:December:2001,49:November:2001,50:November:2001,51:November:2001,52:October:2001,53:October:2001,54:October:2001,55:October:2001,56:September:2001,57:September:2001,58:August:2001,59:August:2001,60:July:2001,61:July:2001,62:June:2001,63:June:2001,64:September:2001,65:October:2001,66:October:2001,67:November:2001,68:December:2001,69:December:2001,70:December:2001,71:January:2002,72:January:2002,73:February:2002,74:March:2002,75:March:2002,76:March:2002,77:April:2002,78:May:2002,79:May:2002,80:June:2002,81:June:2002,82:July:2002,83:July:2002,84:August:2002,85:July:2005,86:June:2001,87:June:2001,88:July:2001,89:July:2001,90:August:2001,91:August:2001,92:September:2001,93:September:2001,94:September:2001,95:October:2001,96:October:2001,97:November:2001,98:November:2001,...,3825:June:2019,3826:November:2020,3827:November:2020,3828:November:2020,3829:December:2020,3830:August:2020,3831:August:2020,3832:September:2020,3833:September:2020,3834:September:2020,3835:October:2020,3836:October:2020,3837:October:2020,3838:January:2021,3839:January:2021,3840:July:2020,3841:July:2020,3842:July:2020,3843:July:2020,3844:August:2020,3845:August:2020,3846:August:2020,3847:August:2020,3848:September:2020,3849:September:2020,3850:September:2020,3851:October:2020,3852:October:2020,3853:October:2020,3854:November:2020,3855:July:2020,3856:July:2020,3857:July:2020,3858:July:2020,3859:August:2020,3860:August:2020,3861:August:2020,3862:September:2020,3863:September:2020,3864:September:2020,3865:October:2020,3866:October:2020,3867:October:2020,3868:November:2020,3869:November:2020,3870:December:2020,3871:December:2020,3872:December:2020,3873:January:2021,3874:January:2021,3875:January:2021,3876:February:2021,3877:February:2021,3878:February:2021,3879:March:2021,3880:March:2021,3881:March:2021,3882:April:2021,3883:April:2021,3884:April:2021,3885:June:2021,3886:June:2020,3887:July:2020,3888:July:2020,3889:July:2020,3890:August:2020,3891:August:2020,3892:August:2020,3893:September:2020,3894:September:2020,3895:September:2020,3896:October:2020,3897:October:2020,3898:October:2020,3899:December:2020,3900:January:2021,3901:January:2021,3902:January:2021,3903:February:2021,3904:April:2021,3905:April:2021,3906:April:2021,3907:May:2021,3908:May:2021,3909:June:2021,3910:June:2020,3911:July:2020,3912:July:2020,3913:July:2020,3914:August:2020,3915:August:2020,3916:August:2020,3917:September:2020,3918:September:2020,3919:September:2020,3920:October:2020,3921:October:2020,3922:October:2020,3923:November:2020,3924:November:2020
0,-3,15.655,14.549,15.049,12.237,14.069,14.178,11.103,10.482,13.578,14.729,14.81,12.85,14.77,15,15.42,12.12,12.14,12.35,12.59,13.17,13.69,14.23,14.78,15.44,16.45,16.65,16.28,17.4,17.37,16.86,16.78,16.51,16.38,14.91,14.71,14.08,14.03,13.13,12.09,11.88,12.19,12.38,12.3,12.37,12.29,12.5,12.6,12.9,13.63,13.96,14.54,14.91,15.65,16.02,16.41,17.3,17.82,17.53,17.19,17.71,16.14,16.84,16.11,15.06,17.183,16.488,15.033,13.423,13.269,12.936,12.698,12.896,12.43,11.964,11.851,11.909,12.079,12.378,13.163,13.582,14.176,15.055,15.663,17.038,17.28,16.539,15.24,15.83,16.67,15.92,17.48,17.2,17.53,17.87,17.54,16.02,15.62,14.94,14.51,...,15.575,12.069,11.719,11.564,10.98,17.155,17.018,16.609,17.163,15.879,15.694,15.317,14.06,11.932,12.024,16.993,15.109,15.051,17.267,17

In [ ]:
# def groupbyVaex(df:vx.dataframe):
#     return df.groupby('z').agg('mean')
# df = vx.open(parquetFile)
df.sample(10).groupby(by='z',agg='mean',progress=True)
# gbPolars = groupbyVaex(df=df)
# pd.testing.assert_frame_equal(gbPandas,gbPolars.to_pandas().set_index('z'))

groupby [####################--------------------] 50.00% estimated time:     0.11s =  0.0m =  0.0h 

In [80]:
df = pl.read_parquet(parquetFile).to_pandas()
printmd('**Pandas**')
%timeit -n 1 -r 1 groupbyPandas(df=df)

**Pandas**

2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [81]:
df = pl.read_parquet(parquetFile)
printmd('**Polars**')
%timeit -n 1 -r 1 groupbyPolars(df=df)

**Polars**

422 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
